In [ ]:
import numpy as np
from simulate.simulate_spatial_cells import sim_hd_cell_spikes, sim_dir_placecell_spikes
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson
from statsmodels.genmod.families.links import identity, log
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from scipy.stats import chi2
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import multivariate_normal
from scipy.io import loadmat
from utils import get_dir_from_xy

import matplotlib

font = {'size'   : 20}
matplotlib.rc('font', **font)  # pass in the font dict as kwargs


In [ ]:
# load some example data
data = loadmat('hipp_data')
xPos = data['xN'].squeeze() * 100  # convert to cm
yPos = data['yN'].squeeze() * 100
samp_freq = np.round(1 / np.diff(data['T'].squeeze())[0])
spikes = data['spikes2'].squeeze()

head_dir = get_dir_from_xy(xPos, yPos) + np.pi

In [ ]:
# simulate a head direction cell
pref_dir = np.pi / 4  # preferred firing direction
precision = 1.  # inverse of width of tuning
max_rate = 15

spikes, c = sim_hd_cell_spikes(head_dir, pref_dir=pref_dir, concentration=precision, max_rate=max_rate, sampling_frequency=samp_freq, return_object=True)

In [ ]:
# get a directional ratemap for the simulated cell

n_bins = 20
bin_edges = np.linspace(0, np.pi*2, n_bins + 1)
bin_size_rad = 2 * np.pi / n_bins

spike_hist, _ = np.histogram(head_dir[spikes], bin_edges)

dir_occupancy, _  = np.histogram(head_dir, bin_edges)
dir_occupancy = dir_occupancy / samp_freq  # convert to seconds

ratemap = spike_hist / dir_occupancy

bin_centres = bin_edges[:-1] + bin_size_rad / 2

width = 2 * np.pi / len(ratemap)
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.bar(bin_centres, ratemap, width=width)

#plt.xlim(0,7)

    # Fitting a poisson-GLM

We will run a poisson-GLM with $\sin(\theta)$ and $\cos(\theta)$ as predictors:$$\log \lambda(t) = \beta_0 + \beta_1 \sin \theta + \beta_2 \cos \theta $$

This can be rewritten as a Von Mises: $$\lambda = \alpha \exp(\kappa \cos(\theta-\phi))$$
with preferred firing rate $\phi = \arctan(\beta_1/\beta_2)$ and concentration (inverse width of tuning curve) $\kappa= \beta_2 \sqrt \left(1+\frac{\beta_1^2}{\beta_2^2} \right)$ and max firing rate $\alpha = \exp(\beta_0) \kappa$


In [ ]:
predictors = df({
    'Intercept': np.ones_like(head_dir),
    'Sin': np.sin(head_dir),
    'Cos': np.cos(head_dir)
})

model = sm.GLM(spikes, predictors, family=Poisson())  # Create the model
results = model.fit()
betas = results.params

In [ ]:
pref_dir_hat = np.arctan(betas[1]/betas[2])  # that looks about right
precision_hat = betas[2] * np.sqrt(1 + (betas[1]/betas[2])**2)  # awesome
max_rate_hat = np.exp(betas[0]+precision_hat) *samp_freq  # cool

print('True max rate {:.2f}, estimated {:.2f}'.format(max_rate, max_rate_hat))
print('True preferred direction {:.2f}, estimated {:.2f} in pi radians'.format(pref_dir / np.pi, pref_dir_hat/np.pi))
print('True precision {:.2f}, estimated {:.2f}'.format(precision, precision_hat))


In [ ]:
new_predictors = df({
    'Intercept': np.ones_like(bin_centres),
    'Sin': np.sin(bin_centres),
    'Cos': np.cos(bin_centres)
})

predicted_rates = results.predict(new_predictors)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

ax.bar(bin_centres, predicted_rates*samp_freq, width=width)
plt.title('Predicted rates')

# Next, simulate a directional place cell

In [ ]:
pref_pos = np.array([0, 70])
pref_dir = np.pi / 2
max_rate = 30
samp_freq = 100

spikes = sim_dir_placecell_spikes(np.array([xPos,yPos]).T, head_dir, pref_pos=pref_pos, pref_dir=pref_dir,
                                  dir_precision=2, max_rate=max_rate, sampling_frequency=samp_freq)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)

up_mask = (head_dir >= 0) & (head_dir < np.pi)
down_mask = (head_dir >= np.pi) & (head_dir < 2*np.pi)

up_spikes = np.copy(spikes)
down_spikes = np.copy(spikes)

up_spikes[down_mask] = False
down_spikes[up_mask] = False

ax1.plot(xPos[up_mask], yPos[up_mask])
ax1.scatter(xPos[up_spikes],yPos[up_spikes], color='r',zorder=20)

ax2.plot(xPos[down_mask], yPos[down_mask])
ax2.scatter(xPos[down_spikes],yPos[down_spikes], color='r',zorder=20)

plt.tight_layout()

In [ ]:
# assert that we don't double count any spikes
assert np.intersect1d(np.flatnonzero(down_spikes), np.flatnonzero(up_spikes)).size == 0

# Now fit a GLM

In [ ]:
predictors = df({
    'Intercept': np.ones_like(xPos),
    'X': xPos,
    'Y': yPos,
    'X2': xPos ** 2,
    'Y2': yPos ** 2,
    'XY': xPos * yPos,
    'Sin': np.sin(head_dir),
    'Cos': np.cos(head_dir)
})

model = sm.GLM(spikes, predictors, family=Poisson())  # Create the model
results = model.fit()
betas = results.params


In [ ]:
pref_dir_hat = np.arctan(betas['Sin']/betas['Cos'])
precision_hat = betas['Cos'] * np.sqrt(1 + (betas['Sin']/betas['Cos'])**2)

print('True preferred direction {:.2f}, estimated {:.2f} in pi radians'.format(pref_dir / np.pi, pref_dir_hat/np.pi))
print('True precision {:.2f}, estimated {:.2f}'.format(precision, precision_hat))

In [ ]:
pref_x_hat = - betas['X'] / (2 * betas['X2'])
pref_y_hat = - betas['Y'] / (2 * betas['Y2'])

print('True field centre ({}, {}), estimated ({} {})'.format(*pref_pos, pref_x_hat, pref_y_hat))

In [ ]:
# how well do we fit the simulated cell with this GLM?
spikeindex = np.where(spikes!=0)[0]
lambda_hat = results.predict(predictors)

Z = [sum(lambda_hat[a:b]) for a, b, in zip(np.hstack((0, spikeindex[:-1])), spikeindex)]


In [ ]:
ecdf = ECDF(Z)
mCDF = 1 - np.exp(-ecdf.x)
N = len(Z)

plt.plot(mCDF, ecdf.y)
plt.plot([0,1], [0,1] + 1.36/np.sqrt(N),'k--', ms = 2)
plt.plot([0,1], [0,1] - 1.36/np.sqrt(N),'k--', ms = 2)
plt.xlabel('Model CDF')                 #Label the axes.
plt.ylabel('Empirical CDF')
plt.title('KS plot of Model')
plt.show()

In [ ]:
results.conf_int()